In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv')

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!pip install -r yolov5/requirements.txt

In [ ]:
import torch
import numpy as np
import pandas as pd
import shutil as sh
import cv2
import os

In [ ]:
image = cv2.imread('/kaggle/input/car-object-detection/data/training_images/vid_4_10020.jpg')

In [ ]:
print(image.shape)

In [ ]:
df.head()

In [ ]:
df['image'] = df['image'].apply(lambda x: x.split('.')[0])
df['x_center'] = (df['xmin'] + df['xmax'])/2/676
df['y_center'] = (df['ymin'] + df['ymax'])/2/380
df['w'] = (df['xmax'] - df['xmin'])/676
df['h'] = (df['ymax'] - df['ymin'])/380
df['class'] = 0
df['status'] = 1

In [ ]:
df.head()

In [ ]:
if not os.path.exists(os.path.join('mydata','images')):
    os.makedirs(os.path.join('mydata','images'))
if not os.path.exists(os.path.join('mydata','labels')):
    os.makedirs(os.path.join('mydata','labels'))
for idx,data in df.groupby('image'):
    row = data[['class','x_center','y_center','w','h']].astype(float).values
    row = row.astype(str)
    with open('mydata/labels/{}.txt'.format(idx),'w+') as f:
        for i in range(len(row)):
            text = ' '.join(row[i])
            f.write(text)
            f.write('\n')
    sh.copy('/kaggle/input/car-object-detection/data/training_images/{}.jpg'.format(idx),
           'mydata/images/{}.jpg'.format(idx))

In [ ]:
with open('yolov5/dataset.yaml','w+') as f:
    f.write('path: ../mydata\n')
    f.write('train: images\n')
    f.write('val: images\n')
    f.write('names: \n')
    f.write(' 0: car')

In [ ]:
cd yolov5

In [ ]:
!python train.py --img 676 --batch 16 --epochs 5 --data dataset.yaml --weight yolov5s.pt --workers 2

In [ ]:
cd ..

In [ ]:
model = torch.hub.load('ultralytics/yolov5','custom', path = 'yolov5/runs/train/exp/weights/last.pt',force_reload = True)

In [ ]:
result = model('/kaggle/input/car-object-detection/data/testing_images/vid_5_26720.jpg')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(np.squeeze(result.render()))
plt.show()

In [ ]:
!pip install imutils
from imutils import paths
list_test = list(paths.list_images('/kaggle/input/car-object-detection/data/testing_images'))

In [ ]:
for i in list_test:
    result = model(i)
    plt.imshow(np.squeeze(result.render()))
    plt.show()